In [11]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime, date
from psx import stocks, tickers
from IPython.display import display
import pandas as pd
import investpy
import yfinance as yf
from matplotlib.dates import relativedelta
from apps.environment.models import Stock
from apps.environment.models.stock import StockBuffer
from apps.environment.models.commodity import Commodity
from apps.environment.models.commodity import CommodityBuffer
import pytz
from os import environ
from math import ceil, floor
import psycopg2
import numpy as np



In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Assuming tickers is your DataFrame


tickers_column = tickers()
# display(tickers_column)
symbols = list(tickers_column["symbol"])
names = list(tickers_column["name"])
len_syms = len(symbols)
# # print(symbols)
hundred_inds = pd.read_csv('KSE100 indx Comps - stock-exchange-kse-100pakistan.csv').to_dict(orient='records')
# print(hundred_inds)
hun_syms = {}

all_syms = {}
for ind in range(len_syms):
    all_syms[symbols[ind]] = names[ind]

the_index = 1
for hun_ind in hundred_inds:
    if hun_ind["SYMBOL"] in all_syms:
        sym_b = hun_ind["SYMBOL"]
        if sym_b not in hun_syms:
            hun_syms[sym_b] = {
                "index": the_index,
                "name": all_syms[sym_b]
            }
            the_index = the_index + 1

db_stcks = {}
all_stcks = Stock.objects.all()
max_indx = len(all_stcks)
for astck in all_stcks:
    db_stcks[astck.symbol] = astck.symbol

new_stcks_list = []
for sym_bl in hun_syms:
    if sym_bl not in db_stcks:
        sym_data = hun_syms[sym_bl]
        new_indx = sym_data["index"]
        if new_indx<=max_indx:
            new_indx = max_indx+1
        max_indx = max_indx+1
        new_stcks_list.append(
            Stock(
                index=sym_data["index"],
                symbol=sym_bl,
                name=sym_data["name"],
                type=100
            )
        )

Stock.objects.bulk_create(new_stcks_list)

In [ ]:


all_stocks = Stock.objects.all()

stock_syms = {}

for a_stock in all_stocks:
    if a_stock.symbol not in stock_syms:
        stock_syms[a_stock.symbol] = {
            "object": a_stock,
            "snapshots": {},
            "last_date": None
        }

snapshots = StockBuffer.objects.select_related("stock").all()
for snapshot in snapshots:
    snp_stck = snapshot.stock
    snp_stck_symbl = snp_stck.symbol
    if snp_stck_symbl in stock_syms:
        snp_data = stock_syms[snp_stck_symbl]
        stck_snps = snp_data["snapshots"]
        if snapshot.captured_at not in stck_snps:
            stck_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not snp_data["last_date"]:
                snp_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for stck_sym, stck_data in stock_syms.items():
    new_snapshots = []
    last_date: datetime = stck_data["last_date"]
    if last_date is None:
        start_from = date(2020, 1, 1)
    else:
        start_from = last_date
    print(last_date, stck_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = stocks(stck_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    stck_snpshts = stck_data["snapshots"]
    the_stock: Stock = stck_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in stck_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(StockBuffer(
                stock = the_stock,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
                change = change,
                volume = data_point.Volume,
                bid_vol = 100000000,
                bid_price = nw_price,
                offer_vol = 100000000,
                offer_price = nw_price,
                open = data_point.Open,
                close = nw_price,
                high = data_point.High,
                low = data_point.Low
            ))
        last_data_point = data_point
    StockBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [ ]:


all_commodities = Commodity.objects.all()

commodity_syms = {}

for a_commodity in all_commodities:
    if a_commodity.symbol not in commodity_syms:
        commodity_syms[a_commodity.symbol] = {
            "object": a_commodity,
            "snapshots": {},
            "last_date": None
        }

snapshots = CommodityBuffer.objects.select_related("commodity").all()
for snapshot in snapshots:
    snp_cmmdty = snapshot.commodity
    snp_cmmdty_symbl = snp_cmmdty.symbol
    if snp_cmmdty_symbl in commodity_syms:
        cmmdty_data = commodity_syms[snp_cmmdty_symbl]
        cmmdty_snps = cmmdty_data["snapshots"]
        if snapshot.captured_at not in cmmdty_snps:
            cmmdty_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not cmmdty_data["last_date"]:
                cmmdty_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for cmmdty_sym, cmmdty_data in commodity_syms.items():
    new_snapshots = []
    last_date: datetime = cmmdty_data["last_date"]
    if last_date is None:
        start_from = date(2019, 12, 30)
    else:
        start_from = date(2019, 12, 30)
        # start_from = last_date
    print(last_date, cmmdty_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = yf.download(cmmdty_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    cmmdty_snpshts = cmmdty_data["snapshots"]
    the_commodity: Commodity = cmmdty_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in cmmdty_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(CommodityBuffer(
                commodity = the_commodity,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
            ))
        last_data_point = data_point
    CommodityBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [32]:
starting_time_step = datetime(year=2023, month=9, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 500, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

In [33]:
the_current_time_step = env_config.get("the_current_time_step")
max_epi_len = 30

__last_time_step = the_current_time_step + relativedelta(days=max_epi_len)
the_current_time_step = pytz.utc.localize(datetime.strptime(str(the_current_time_step), '%Y-%m-%d %H:%M:%S'))
__last_time_step = pytz.utc.localize(datetime.strptime(str(__last_time_step), '%Y-%m-%d %H:%M:%S'))
str_time_step = str(the_current_time_step)

str_last_time_step = str(__last_time_step)
db_params = env_config.get("db_params")
db_conn = psycopg2.connect(**db_params)
cursor = db_conn.cursor()

stcks_query = """
    SELECT *
    FROM stocks
"""
cursor.execute(stcks_query)
stck_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stck_df = pd.DataFrame(stck_data, columns=column_names) 
# stck_df = stck_df.to_dict(orient='records')

stcks_buffer_query = f"""
    SELECT stocks_buffers.*, stocks.index
    FROM stocks_buffers 
    JOIN stocks on stocks.id = stocks_buffers.stock_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
    AND stocks.index = 4
"""
cursor.execute(stcks_buffer_query)
stcks_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stcks_buffer_df = pd.DataFrame(stcks_buffer_data, columns=column_names)

# stcks_buffer_df = stcks_buffer_df.to_dict(orient='records')

cmmdties_buffer_query = f"""
    SELECT commodities_buffers.*, commodities.index
    FROM commodities_buffers 
    JOIN commodities on commodities.id = commodities_buffers.commodity_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(cmmdties_buffer_query)
cmmdties_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_buffer_df = pd.DataFrame(cmmdties_buffer_data, columns=column_names)
# cmmdties_buffer_df = cmmdties_buffer_df.to_dict(orient='records')

cmmdties_query = """
    SELECT *
    FROM commodities
"""
cursor.execute(cmmdties_query)
cmmdties_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_df = pd.DataFrame(cmmdties_data, columns=column_names) 

In [34]:


# Specify the date range
start_date = the_current_time_step
end_date = the_current_time_step + relativedelta(days=max_epi_len)

# Create a list of working days within the date range
working_days = pd.date_range(start=start_date, end=end_date, freq='B')

# Repeat rows for each index from 1 to 100
index_range = range(4, 5)

# Create a DataFrame with the specified structure
stcks_data = {'captured_at': [], 'change': [], 'index': []}

for date in working_days:
    for idx in index_range:
        stcks_data['captured_at'].append(date)
        stcks_data['change'].append(0)
        stcks_data['index'].append(idx)

stcks_working_day_df = pd.DataFrame(stcks_data)

cmmdts_data = {'captured_at': [], 'index': []}

for date in working_days:
    for idx in index_range:
        cmmdts_data['captured_at'].append(date)
        cmmdts_data['index'].append(idx)

cmmdts_working_day_df = pd.DataFrame(cmmdts_data)

# Print the resulting DataFrame


In [35]:
display(stcks_buffer_df)

,id,captured_at,price_snapshot,change,volume,bid_vol,bid_price,offer_vol,offer_price,stock_id,close,high,low,open,index
0,52652,2023-09-12 00:00:00+00:00,21255.0,-0.071834,20.0,100000000.0,21255.0,100000000.0,21255.0,4,21255.0,21255.0,21255.0,21255.0,4
1,52653,2023-09-15 00:00:00+00:00,21255.0,0.000000,60.0,100000000.0,21255.0,100000000.0,21255.0,4,21255.0,21255.0,21255.0,21255.0,4
2,52654,2023-09-21 00:00:00+00:00,20100.0,-0.054340,60.0,100000000.0,20100.0,100000000.0,20100.0,4,20100.0,20355.0,20100.0,20355.0,4
3,52655,2023-09-27 00:00:00+00:00,20550.0,0.022388,20.0,100000000.0,20550.0,100000000.0,20550.0,4,20550.0,20550.0,20550.0,20550.0,4
4,52656,2023-09-28 00:00:00+00:00,20550.0,0.000000,80.0,100000000.0,20550.0,100000000.0,20550.0,4,20550.0,20550.0,20550.0,20550.0,4


In [36]:
# id	captured_at	price_snapshot	change	volume	bid_vol	bid_price	offer_vol	offer_price	stock_id	close	high	low	open	index

stocks_fields_list = ["id", "price_snapshot", "volume", "bid_vol",	"bid_price", "offer_vol",	"offer_price",	"stock_id",	"close",	"high",	"low",	"open",]

stcks_df1 = stcks_buffer_df
stcks_df2 = stcks_working_day_df

# Merge dataframes on 'captured_at' and 'index' and keep only the rows present in stcks_df2
stcks_merged_df = pd.merge(stcks_df2, stcks_df1, on=['captured_at', 'index'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)


stcks_concatenated_df = pd.concat([stcks_df1, stcks_merged_df], ignore_index=True)

# Print the concatenated dataframe

# Sort the DataFrame by 'index' and 'captured_at'
stcks_concatenated_df = stcks_concatenated_df.sort_values(by=['index', 'captured_at'])

# Forward fill NaN values based on 'index'

for field in stocks_fields_list:
    stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
    stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')


# Display the upcaptured_atd DataFrame

stcks_concatenated_df['change'].fillna(stcks_concatenated_df['change_x'], inplace=True)

# Drop the 'change_x' column if you no longer need it
stcks_concatenated_df.drop(columns=['change_x', 'change_y'], inplace=True)
display(stcks_concatenated_df)

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_61293/2518332673.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_61293/2518332673.py:23: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')


,id,captured_at,price_snapshot,change,volume,bid_vol,bid_price,offer_vol,offer_price,stock_id,close,high,low,open,index
5,52652.0,2023-09-01 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
6,52652.0,2023-09-04 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
7,52652.0,2023-09-05 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
8,52652.0,2023-09-06 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
9,52652.0,2023-09-07 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
10,52652.0,2023-09-08 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
11,52652.0,2023-09-11 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
0,52652.0,2023-09-12 00:00:00+00:00,21255.0,-0.071834,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
12,52652.0,2023-09-13 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4
13,52652.0,2023-09-14 00:00:00+00:00,21255.0,0.000000,20.0,100000000.0,21255.0,100000000.0,21255.0,4.0,21255.0,21255.0,21255.0,21255.0,4


In [37]:
# id	captured_at	price_snapshot	change	volume	bid_vol	bid_price	offer_vol	offer_price	stock_id	close	high	low	open	index

commodities_fields_list = ["id", "price_snapshot", "commodity_id"]

cmmdts_df1 = cmmdties_buffer_df
cmmdts_df2 = cmmdts_working_day_df

# Merge dataframes on 'captured_at' and 'index' and keep only the rows present in cmmdts_df2
cmmdts_merged_df = pd.merge(cmmdts_df2, cmmdts_df1, on=['captured_at', 'index'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)


cmmdts_concatenated_df = pd.concat([cmmdts_df1, cmmdts_merged_df], ignore_index=True)

# Print the concatenated dataframe

# Sort the DataFrame by 'index' and 'captured_at'
cmmdts_concatenated_df = cmmdts_concatenated_df.sort_values(by=['index', 'captured_at'])

# Forward fill NaN values based on 'index'

for field in commodities_fields_list:
    cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='ffill')
    cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='bfill')


# Display the upcaptured_atd DataFrame

display(cmmdts_concatenated_df)

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_61293/1537844305.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_61293/1537844305.py:23: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='bfill')


,id,captured_at,price_snapshot,commodity_id,index
0,924.0,2023-09-01 00:00:00+00:00,1939.800049,1.0,1
1,925.0,2023-09-05 00:00:00+00:00,1926.199951,1.0,1
2,926.0,2023-09-06 00:00:00+00:00,1918.099976,1.0,1
3,927.0,2023-09-07 00:00:00+00:00,1917.500000,1.0,1
4,928.0,2023-09-08 00:00:00+00:00,1918.400024,1.0,1
5,929.0,2023-09-11 00:00:00+00:00,1923.300049,1.0,1
6,930.0,2023-09-12 00:00:00+00:00,1911.300049,1.0,1
7,931.0,2023-09-13 00:00:00+00:00,1909.099976,1.0,1
8,932.0,2023-09-14 00:00:00+00:00,1910.000000,1.0,1
9,933.0,2023-09-15 00:00:00+00:00,1923.699951,1.0,1


In [ ]:
# Sample data for illustration
data1 = {'date': ['2023-01-01', '2023-01-06', '2023-01-02', '2023-01-03'],
         'id': [1, 1, 2, 3],
         'value': ['A', "Yo", 'B', 'C'],
         'valuea': ['a', "y", 'b', 'c'],
         'change': [1,2,3,4],
         }
df1 = pd.DataFrame(data1)

data2 = {'date': ['2023-01-01', '2023-01-04', '2023-01-04', '2023-01-03'],
         'id': [1, 1, 2, 3],
         'change': [0,0,0,0],
         }
df2 = pd.DataFrame(data2)

# Merge dataframes on 'date' and 'id' and keep only the rows present in df2
merged_df = pd.merge(df2, df1, on=['date', 'id'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)


concatenated_df = pd.concat([df1, merged_df], ignore_index=True)

# Print the concatenated dataframe

# Sort the DataFrame by 'id' and 'date'
concatenated_df = concatenated_df.sort_values(by=['id', 'date'])

# Forward fill NaN values based on 'id'
concatenated_df['value'] = concatenated_df.groupby('id')['value'].fillna(method='ffill')
concatenated_df['value'] = concatenated_df.groupby('id')['value'].fillna(method='bfill')

concatenated_df['valuea'] = concatenated_df.groupby('id')['valuea'].fillna(method='ffill')
concatenated_df['valuea'] = concatenated_df.groupby('id')['valuea'].fillna(method='bfill')

# Display the updated DataFrame

concatenated_df['change'].fillna(concatenated_df['change_x'], inplace=True)

# Drop the 'change_x' column if you no longer need it
concatenated_df.drop(columns=['change_x', 'change_y'], inplace=True)
print(concatenated_df)

In [ ]:
for dyIndx in range(1,3):
    print(dyIndx)

In [13]:
start_from = date(2023, 1, 1)
end_date = date(2023, 2, 1)
data = stocks("AIRLINK", start=start_from, end=end_date)
display(data)

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-02,29.00,29.49,28.51,28.88,88500.0
2023-01-03,28.51,28.51,26.80,26.92,427500.0
2023-01-04,27.20,27.20,26.56,26.86,68500.0
2023-01-05,26.72,27.23,26.60,26.76,432000.0
2023-01-06,26.31,27.18,25.81,25.99,129000.0
2023-01-09,25.87,25.87,25.26,25.36,60000.0
2023-01-10,26.24,26.24,25.42,25.57,74500.0
2023-01-11,25.51,25.60,25.26,25.28,43000.0
2023-01-12,25.46,26.00,25.35,25.39,111500.0
